In [ ]:
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
import json
import re


storage_client = storage.Client()
bq_client = bigquery.Client()

bucket = storage_client.bucket('boosta-task')

blobs = bucket.list_blobs(prefix='')

for blob in blobs:
  if blob.name.endswith('.json'):
    file_name = blob.name.split('/')[-1].replace('.json', '')
    match = re.match(r'^([a-zA-Z0-9]+_[a-zA-Z0-9]+)', file_name)
    if match:
      tablename = match.group(1)
    content = blob.download_as_text()
    data = json.loads(content)['data']['2024-06']
    df = pd.DataFrame(data)
    table_id = f'crack-node-462719-n0.raw_layer.{tablename}'
    job = bq_client.load_table_from_dataframe(df, table_id, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    job.result()

